In [1]:
import os
import sys

# Obtener el directorio actual de la notebook
notebook_dir = os.getcwd()

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Agregar el directorio padre al path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from libs.config.config_variables import (
    MAX_WORKERS,
    TIMEOUT_API_REQUEST,
    CATALOG_CSV_DIR,
)

from modules.catalog_scraper.seismic_scraper import SeismicScraper
from modules.downloader.records_downloader import SeismicDownloader

# 0. Configuración básica de catálogos

In [3]:
# Límites geográficos (Perú y alrededores)
MIN_LAT, MAX_LAT = -20.0, 0.0
MIN_LON, MAX_LON = -85.0, -68.0

# Año de inicio del scraping
START_YEAR = 1400

# 1. Ejecución de scraping de catálogos


## 1.1 Carga de datos históricos

In [4]:
# Registrar los catálogos csv
historical_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST, csv_base_path=CATALOG_CSV_DIR
)
results = historical_scraper.scrape_events_from_csv(
    catalogs=["gcmt", "isc-gem", "sara"],
    # start_year=START_YEAR,
    # min_lat=MIN_LAT,
    # max_lat=MAX_LAT,
    # min_lon=MIN_LON,
    # max_lon=MAX_LON,
)


2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events_from_csv
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 1 archivos CSV para GCMT
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para ISC-GEM
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para SARA
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: 5 archivos con 40 workers
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: GCMT - gcmt_data.csv
2025-12-07 11:43:33,687 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-cat.csv
2025-12-07 11:43:33,721 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-suppl.csv
2025-12-07 11:43:33,722 [INFO] modules.ca

In [5]:
historical_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|     178748      | 178748 |      0       |    1    |
+-----------------+--------+--------------+---------+


## 1.2 Actualización de datos recientes

In [6]:
recent_data_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST
)

results = recent_data_scraper.scrape_events(
    start_year=2000,
    min_lat=MIN_LAT,
    max_lat=MAX_LAT,
    min_lon=MIN_LON,
    max_lon=MAX_LON,
    catalogs=["usgs", "isc", "igp"],
)

2025-12-07 11:43:46,278 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events
2025-12-07 11:43:46,279 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: 78 tareas con 40 workers
2025-12-07 11:43:46,281 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2000
2025-12-07 11:43:46,282 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2001
2025-12-07 11:43:46,298 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2002
2025-12-07 11:43:46,299 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2003
2025-12-07 11:43:46,300 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2004
2025-12-07 11:43:46,315 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2005
2025-12-07 11:43:46,315 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2006
2025-12-07 11:43:46,315 [INFO] m

In [7]:
recent_data_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|      15702      | 15599  |     103      |   15    |
+-----------------+--------+--------------+---------+


## 1.3 Descarga de registros acelerométricos

In [ ]:
PARALLEL_STATIONS = 20
CATALOG_PARSER_CONFIG = {
    "igp": '2000-01-01',
    # "usgs": '2025-12-28',
    # "emsc": '2025-12-28',
}

In [9]:
downloader = SeismicDownloader()

downloader.process_multiple_catalogs(
    catalogs=CATALOG_PARSER_CONFIG,
    max_workers=MAX_WORKERS,
    parallel_stations=PARALLEL_STATIONS,
)

2025-12-07 11:44:03,388 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_multiple_catalogs
2025-12-07 11:44:03,389 [INFO] modules.downloader.records_downloader: Procesando 1 catálogos
2025-12-07 11:44:03,389 [INFO] modules.downloader.records_downloader:   - IGP: desde 2025-11-01
2025-12-07 11:44:03,389 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_catalog
2025-12-07 11:44:03,392 [INFO] modules.downloader.records_downloader: Procesando catálogo: IGP desde 2025-11-01
2025-12-07 11:44:03,395 [INFO] modules.downloader.records_downloader: Encontrados 73 eventos para procesar
2025-12-07 11:44:03,397 [INFO] libs.database.seismic_records: Inicio de sesión de sincronización (record_id=1, catalog_id=122)
2025-12-07 11:44:03,484 [WARNING] modules.downloader.records_downloader: 2025-0724: Procesamiento fallido
2025-12-07 11:44:03,499 [WARNING] modules.downloader.records_downloader: 2025-0723: Procesamiento

{'success': True,
 'catalogs_processed': 1,
 'results': {'igp': {'success': True,
   'catalog': 'igp',
   'sync_id': 1,
   'status': 'completed_with_errors',
   'total_events': 73,
   'successful_events': 10,
   'failed_events': 63,
   'total_stations': 238,
   'total_samples': 4178613,
   'total_processing_time': 149.09,
   'avg_processing_time': 2.04,
   'error_summary': '2025-0724: Procesamiento fallido; 2025-0723: Procesamiento fallido; 2025-0726: Procesamiento fallido; 2025-0722: Procesamiento fallido; 2025-0728: Procesamiento fallido...',
   'timestamp': '2025-12-07T11:44:23.065549'}},
 'summary': {'total_events': 73,
  'successful_events': 10,
  'failed_events': 63,
  'total_stations': 238,
  'total_samples': 4178613},
 'timestamp': '2025-12-07T11:44:23.066560'}

In [10]:
downloader.metadata

+----------+-----------------------+---------+---------+----------+----------+----------------+------------+----------+------------------+------------------------+---------------------------------------------------------------------------------------------------------+
| Catálogo |        Estado         | Sync ID | Eventos | Exitosos | Fallidos | Tasa Éxito (%) | Estaciones | Muestras | Tiempo Total (s) | Tiempo Prom/Evento (s) |                                            Errores (resumen)                                            |
+----------+-----------------------+---------+---------+----------+----------+----------------+------------+----------+------------------+------------------------+---------------------------------------------------------------------------------------------------------+
|   IGP    | completed_with_errors |    1    |   73    |    10    |    63    |      13.7      |    238     | 4178613  |      149.1       |          2.04          | 2025-0724: Procesamiento f